# **GIS × Python Tutorial 5.1 ~ geopandas ことはじめ GeoDataFrameの基本 ~**

<br>

## **はじめに**
この記事は「GIS × Python Tutorial」の関連記事です。
今回は`geopandas.GeoDataFrame`の基本について解説します。pythonを使用する方であればpandasを使用した事がある方が多いかと思いますので、分からない部分はgeopandasドキュメントを見て行けば躓かずに理解できるかと思います。またgeopandasのgeometryはshapelyのgeometryオブジェクトが入力されるので、個別のメソッドはshapelyの公式ドキュメント、あるいは前回の記事を参考にしてください。

https://zenn.dev/daidai_daitai/articles/968e08b495f9e2

<br>

## **geopandas とは**
**geopandas**は、pandasの拡張であり地理空間情報を扱いやすくした DataFrame を操作する事が出来ます。ドキュメントも非常に読みやすいと思いますので、このシリーズを読んでもっと詳しく知りたい方はドキュメントを読んでみてください。またチュートリアルも充実しているようなので GitHub のリンクも貼っておきます。また今回の実行環境や

https://geopandas.org/en/stable/docs.html

https://github.com/geopandas/geopandas/tree/main/doc/source/gallery


<br>

## **Install geopandas**

https://geopandas.org/en/stable/getting_started.html

<br>

## **コード実行の準備**

### **Import**

In [137]:
# 今回使用するライブラリのインポート
import fiona
import geopandas as gpd
from matplotlib import pyplot as plt
import numpy as np
import shapely
plt.style.use('ggplot')

### **データの作成**
適当にデータを作成します。

In [142]:
# 弘前城の位置
x1 = 140.46362773837438
y1 = 40.60790364299233
point1 = shapely.Point(x1, y1)

# 弘前城二の丸の位置
x2 = 140.46539395937307
y2 = 40.60780032475679
point2 = shapely.Point(x2, y2)

# ランダムな点群を作成
length = 200
alphabet = 'ABCDE'
xs = np.random.normal(x1, abs(x1 - x2), length)
ys = np.random.normal(y1, abs(y1 - y2), length)
points = [shapely.Point(_x, _y) for _x, _y in zip(xs, ys)]
# アルファベットからランダムに値を取得
indexes = np.random.randint(0, len(alphabet), length)
codes = [alphabet[idx] for idx in indexes]
# 適当な値を作成
values1 = np.random.normal(100, 20, length)

<br>

## **GeoDataFrameの作成**
`geopandas.GeoDataFrame`の作成はほとんど`pandas.DataFrame`と同じです。違うのは **geometry** と **crs** を設定する点です。geometry には `shapely.geometry.XXX` の Geometry を List などの Iterable な型に入れて渡します。crs には EPSG コードや WKT-CRS文字列等の座標系が識別できるものを渡します。

In [24]:
IN_EPSG = 'EPSG:4326'

gdf = (
    gpd.GeoDataFrame(
        data={'CODE': codes},
        # 今回はgeometryにList[shapely.Point]を渡す。
        geometry=points,
        crs=IN_EPSG
    )
)

print(f"shape: {gdf.shape}")
gdf.head()

shape: (200, 2)


,CODE,geometry
0,A,POINT (140.46436 40.60803)
1,E,POINT (140.46494 40.60793)
2,C,POINT (140.46419 40.60786)
3,D,POINT (140.46323 40.60783)
4,C,POINT (140.45861 40.60794)


## **geometry**
geometry の列を見てみましょう。geometry の列は`geopandas.geoseries.GeoSeries`になっているのが確認できます。この状態の時には geopadnas は地理的空間的なメソッドを扱う事が出来る様になっています。pandas のメソッドを扱ったりするとこれが外れたりするので、エラーが出たら確認してみましょう。

In [42]:
print(f"""
<< dtypes >>
{gdf.dtypes}

<< series type >>
CODE = {type(gdf['CODE'])}
geometry = {type(gdf.geometry)}
""")


<< dtypes >>
CODE          object
geometry    geometry
dtype: object

<< series type >>
CODE = <class 'pandas.core.series.Series'>
geometry = <class 'geopandas.geoseries.GeoSeries'>



geometry が設定されていない場合は以下の様にエラーが出ます。

In [65]:
_gdf = gpd.GeoDataFrame(data={'CODE': codes, 'SHAPE': points})
_gdf.geometry

AttributeError: You are calling a geospatial method on the GeoDataFrame, but the active geometry column to use has not been set. 
There are no existing columns with geometry data type. You can add a geometry column as the active geometry column with df.set_geometry. 

geometry を設定してみましょう。実は geometry の列名は何でも構いません。

In [70]:
_gdf.set_geometry('SHAPE', inplace=True)

print(f"""
<< dtypes >>
{_gdf.dtypes}

<< series type >>
CODE = {type(_gdf['CODE'])}
geometry = {type(_gdf.geometry)}
""")


<< dtypes >>
CODE       object
SHAPE    geometry
dtype: object

<< series type >>
CODE = <class 'pandas.core.series.Series'>
geometry = <class 'geopandas.geoseries.GeoSeries'>



### **CRSの確認**
crs を設定していれば`GeodataFrame.crs`のメソッドでCRSを確認する事が出来ます。

In [77]:
gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

wkt-crs も見てみます。

In [83]:
print(gdf.crs.to_wkt(pretty=True))

GEOGCRS["WGS 84",
    ENSEMBLE["World Geodetic System 1984 ensemble",
        MEMBER["World Geodetic System 1984 (Transit)"],
        MEMBER["World Geodetic System 1984 (G730)"],
        MEMBER["World Geodetic System 1984 (G873)"],
        MEMBER["World Geodetic System 1984 (G1150)"],
        MEMBER["World Geodetic System 1984 (G1674)"],
        MEMBER["World Geodetic System 1984 (G1762)"],
        MEMBER["World Geodetic System 1984 (G2139)"],
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]],
        ENSEMBLEACCURACY[2.0]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["geodetic longitude (Lon)",east,
            ORDER[2],
            ANGLEUNIT["degree",0.0174532925199433]],
    USAGE[
        SCOPE["Horizontal component of 3D system."],
        AREA["World."],
      

EPSGコードも出力してみましょう。

In [87]:
print(gdf.crs.to_epsg())

4326


### **CRSが設定されていない場合**
CRSが設定されていない場合は設定する事が可能です。

In [146]:
gdf.crs = None
print(f"EPSG 1: {gdf.crs}")

gdf.set_crs(IN_EPSG, inplace=True)
print(f"EPSG 2: {gdf.crs.to_epsg()}")

EPSG 1: None
EPSG 2: 4326


<br>

## **投影変換**

geopandas での投影変換は非常に簡単です。しかし、CRS が設定されていない場合はエラーになるので、上で書いた様にCRSは確認する様にしましょう。

In [96]:
OUT_EPSG = 'EPSG:6678'
gdf_jgd = gdf.to_crs(OUT_EPSG)


print(f"""
IN_EPSG: {gdf.crs.to_epsg()}
OUT_EPSG: {gdf_jgd.crs.to_epsg()}
""")


IN_EPSG: 4326
OUT_EPSG: 6678



### UTM座標系の推定
`estimate_utm_crs`メソッドで UTM 座標系を推定する事が出来ます。都道府県を跨ぐような中規模のデータセットを使用する場合には非常に便利です。

`estimate_utm_crs`は "datum_name" を指定する事が出来ます。デフォルトは 'WGS 84' ですがいくつか見てみましょう。

In [160]:
gdf.estimate_utm_crs()

<Projected CRS: EPSG:32654>
Name: WGS 84 / UTM zone 54N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 138°E and 144°E, northern hemisphere between equator and 84°N, onshore and offshore. Japan. Russian Federation.
- bounds: (138.0, 0.0, 144.0, 84.0)
Coordinate Operation:
- name: UTM zone 54N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [161]:
gdf.estimate_utm_crs(datum_name='JGD2000')

<Projected CRS: EPSG:3100>
Name: JGD2000 / UTM zone 54N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Japan - between 138°E and 144°E, onshore and offshore.
- bounds: (138.0, 17.63, 144.0, 46.05)
Coordinate Operation:
- name: UTM zone 54N
- method: Transverse Mercator
Datum: Japanese Geodetic Datum 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [162]:
gdf.estimate_utm_crs(datum_name='JGD2011')

<Projected CRS: EPSG:6691>
Name: JGD2011 / UTM zone 54N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Japan - between 138°E and 144°E, onshore and offshore.
- bounds: (138.0, 17.63, 144.0, 46.05)
Coordinate Operation:
- name: UTM zone 54N
- method: Transverse Mercator
Datum: Japanese Geodetic Datum 2011
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

これを使用して投影変換してみましょう。

In [165]:
wkt_crs = gdf.estimate_utm_crs(datum_name='JGD2011')
gdf_utm = gdf.to_crs(wkt_crs)

print(f"EPSG: {gdf_utm.crs.to_epsg()}")

EPSG: 6691


<br>

## **shapely のメソッド**
GeoSeries では shapely のメソッドを使用する事が出来ます。これが geopandas が強力なライブラリーである 1つの理由です。

In [126]:
# 各Pointから10mのBufferを作成
print(gdf_jgd.geometry.buffer(10))

0      POLYGON ((-31213.614 67574.607, -31213.663 675...
1      POLYGON ((-31164.385 67563.137, -31164.433 675...
2      POLYGON ((-31228.231 67556.076, -31228.280 675...
3      POLYGON ((-31309.522 67553.114, -31309.570 675...
4      POLYGON ((-31700.033 67566.941, -31700.081 675...
                             ...                        
195    POLYGON ((-31255.266 67560.511, -31255.314 675...
196    POLYGON ((-31237.935 67569.198, -31237.983 675...
197    POLYGON ((-31643.755 67589.807, -31643.803 675...
198    POLYGON ((-31502.709 67561.446, -31502.757 675...
199    POLYGON ((-31485.363 67565.156, -31485.412 675...
Length: 200, dtype: geometry


In [127]:
# 弘前城の位置を入力したGeoDataFrameを作成
base_gdf_jgd = (
    gpd.GeoDataFrame(
        data={'CODE': ['BASE']}, 
        geometry=[point1], 
        crs=IN_EPSG
    )
    .to_crs(OUT_EPSG)
)

# 各Pointと弘前城との距離を計測
gdf_jgd['from_base'] = gdf_jgd.distance(base_gdf_jgd.geometry.iloc[0])

print(gdf_jgd.head())

  CODE                      geometry   from_base
0    A  POINT (-31223.614 67574.607)   63.640951
1    E  POINT (-31174.385 67563.137)  111.391292
2    C  POINT (-31238.231 67556.076)   47.761971
3    D  POINT (-31319.522 67553.114)   34.646377
4    C  POINT (-31710.033 67566.941)  424.323105


<br>

## **データの出力**

今回は **geojson** で出力する場合と **geopackege** で出力する方法の 2種類を見てみましょう。

 - geojson はファイル名のみ指定すれば保存できます。

 - geopackage はファイル名の他に Layer 名を指定する必要があります。

In [131]:
# geojsonで出力する場合
OUT_FILE_GEOJSON = r'../datasets/session5/Hirosaki.geojson'
gdf.to_file(OUT_FILE_GEOJSON, driver='GeoJSON')

# geopackegeに出力する場合
OUT_FILE_GEOPACKEGE = r'../datasets/session5/Hirosaki.gpkg'

LAYER1 = 'RANDOM_DATA'
gdf.to_file(OUT_FILE_GEOPACKEGE, layer=LAYER1, driver='GPKG')

LAYER2 = 'BasePoints'
gdf.to_file(OUT_FILE_GEOPACKEGE, layer=LAYER2, driver='GPKG')

<br>

## **データの読み込み**

geojson の読み込みは非常に簡単ですが、geopackage は Layer 名を指定する必要があります。

In [132]:
_ = gpd.read_file(OUT_FILE_GEOJSON)
print(_.head())

  CODE                    geometry
0    A  POINT (140.46436 40.60803)
1    E  POINT (140.46494 40.60793)
2    C  POINT (140.46419 40.60786)
3    D  POINT (140.46323 40.60783)
4    C  POINT (140.45861 40.60794)


In [139]:
# fionaでgeopackageに保存されているLayer名を取得
layers = fiona.listlayers(OUT_FILE_GEOPACKEGE)
print(f"Layers: {layers}")

Layers: ['RANDOM_DATA', 'BasePoints']


In [140]:
gdf = gpd.read_file(OUT_FILE_GEOPACKEGE, layer=layers[0])
print(gdf.head())

  CODE                    geometry
0    A  POINT (140.46436 40.60803)
1    E  POINT (140.46494 40.60793)
2    C  POINT (140.46419 40.60786)
3    D  POINT (140.46323 40.60783)
4    C  POINT (140.45861 40.60794)
